In [1]:
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

# Interpolation Loop

In [7]:
import cv2
import os

def resize_and_save(image_path, output_folder, downscale_size, down_interpolation, up_interpolation):

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Load the original image
    original_image = cv2.imread(image_path)

    # Store the original size
    original_size = (original_image.shape[1], original_image.shape[0])  # (width, height)

    # Downscale the image
    downscaled_image = cv2.resize(original_image, downscale_size, interpolation=down_interpolation)

    # Upscale the image back to the original size
    upscaled_image = cv2.resize(downscaled_image, original_size, interpolation=up_interpolation)

    # Extract the filename and create the save path
    filename = os.path.basename(image_path)
    save_path = os.path.join(output_folder, filename)

    # Save the upscaled image
    cv2.imwrite(save_path, upscaled_image)

# Example usage
image_folder = r"\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Interpolation\Original_Images"
output_base_folder = r"\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Interpolation\Comparision_256x256"
downscale_size = (256, 256)  # Example size

# Define the interpolation methods
interpolations = {
    "nearest": cv2.INTER_NEAREST,
    "bilinear": cv2.INTER_LINEAR,
    "bicubic": cv2.INTER_CUBIC,
    "lanczos": cv2.INTER_LANCZOS4,
    "area": cv2.INTER_AREA
}

# Iterate through all combinations of interpolation methods
for down_name, down_method in interpolations.items():
    for up_name, up_method in interpolations.items():
        if down_name != up_name:  # Skip identical method pairs
            combination_folder = f"{down_name}_to_{up_name}"
            output_folder = os.path.join(output_base_folder, combination_folder)

            # Process all images in the folder
            for filename in os.listdir(image_folder):
                if filename.endswith(".png"):
                    image_path = os.path.join(image_folder, filename)
                    resize_and_save(image_path, output_folder, downscale_size, down_method, up_method)

# Comparision calculate the difference in NF/FF

In [8]:
# read the images 
import imageio.v2 as iio
# importing matplotlib modules
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from LightPipes import *
import math
from skimage.color import rgb2gray

In [9]:
def mean_value_of_the_intenstiy_difference(img_orginal,img_256):
    # change the image to grayscale
    img_orginal = rgb2gray(img_orginal)
    img_256 = rgb2gray(img_256)
   

    #change the phase profile to phase value and reverse the phase mask
    phase_orginal=(255-img_orginal)/255*np.pi*2-np.pi
    P256=(255-img_256)/255*np.pi*2-np.pi        # P256 phase information
    size=10*mm
    wavelength=1.03*um
    N=512
    #creat an input field 
    F=Begin(size,wavelength,N)
    F=GaussBeam(F, 4.2*mm) 
    #Phi=Phase(F) #Gaussian input field
    #I=Intensity(F,1)
    # fig=plt.figure(figsize=(10,6))
    # ax1 = fig.add_subplot(131)
    # ax2 = fig.add_subplot(132)
    # ax1.imshow(I,cmap='hot'); ax1.axis('off'); ax1.set_title('Intensity'); 
    # ax2.imshow(Phi,cmap='gray'); ax2.axis('off'); ax2.set_title('Phase')

    # generate the far field 512
    f=300*mm
    Phi=phase_orginal
    #print(np.shape(phase_orginal))

    F_orginal= MultPhase(F, Phi)

    P_lens=Lens(F_orginal, f) # after lense

    F_focus=Fresnel(P_lens,300*mm)  #f ar field 
    F_defocus=Fresnel(P_lens,150*mm)  # near field 

    #focus
    #Phi_focus=Phase(F_focus)
    I_focus=Intensity(F_focus,1)

    # defocus
    #Phi_defocus=Phase(F_defocus)
    I_defocus=Intensity(F_defocus,1)

    # --> calculatet the NF and FF form the original Phasemask

    # generate the far field 256
    Phi=P256 # P64 P128
    F256= MultPhase(F, Phi)
    P_lens=Lens(F256, f) #F64#P F128

    F_focus256=Fresnel(P_lens,300*mm)  #far field 
    F_defocus256=Fresnel(P_lens,150*mm)  #near field 

    #focus
    #Phi_focus256=Phase(F_focus256)
    I_focus256=Intensity(F_focus256,1)

    # defocus
    #Phi_defocus256=Phase(F_defocus256)
    I_defocus256=Intensity(F_defocus256,1)

    #64 and orginal ( defocus)
    I_diff256=I_defocus256-I_defocus
    np.mean(I_diff256)

    # --> calculate th NF and FF based on 256 Phasemask at the moment only the nearfield --<defocus 

    # calculate the mean value of the intenstiy difference MAE maybe change to MSE between
    M3=np.mean(np.abs(I_diff256))


    return M3

In [10]:
# Base folder for comparisons
comparision_base_folder = r"\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Interpolation\Comparision_256x256"

# Initialize the DataFrame
columns = ['Method'] + [f'Image {i+1}' for i in range(len(os.listdir(os.path.join(comparision_base_folder, next(os.walk(comparision_base_folder))[1][0]))))]
df = pd.DataFrame(columns=columns)

# Loop through all method folders
for method_folder in os.listdir(comparision_base_folder):
    method_path = os.path.join(comparision_base_folder, method_folder)
    if os.path.isdir(method_path):
        mse_values = [method_folder]  # First column for the method name
        
        # Loop through all images in the folder
        for image_filename in os.listdir(method_path):
            if image_filename.endswith(".png"):
                # Paths to the images
                original_image_path = os.path.join(r"\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Interpolation\Original_Images", image_filename)
                processed_image_path = os.path.join(method_path, image_filename)
                
                # Load the images
                original_image = iio.imread(original_image_path)
                processed_image = iio.imread(processed_image_path)
                
                # Calculate the MSE and add it to the list
                mse = mean_value_of_the_intenstiy_difference(original_image, processed_image)
                mse_values.append(mse)
        
        # Add the MSE values to the DataFrame
        df.loc[len(df)] = mse_values

# Save the DataFrame as a CSV file
output_csv_path = os.path.join(comparision_base_folder, "mean_value_of_the_intenstiy_difference_256x256.csv")
df.to_csv(output_csv_path, index=False)
print(f"CSV saved at: {output_csv_path}")

CSV saved at: \\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Interpolation\Comparision_256x256\mean_value_of_the_intenstiy_difference_256x256.csv


# Results

In [12]:
# 64x64 architecture
import pandas as pd
pd.set_option('display.float_format', '{:.20f}'.format)
file_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datasets\Test_Resize\Comparision_64x64\mean_value_of_the_intenstiy_difference_64x64.csv'

dataframe = pd.read_csv(file_path)
dataframe['Averrage'] = dataframe.iloc[:, 1:].mean(axis=1)
dataframe.sort_values(by='Averrage').head(25)

,Method,Image 1,Image 2,Image 3,Image 4,Image 5,Averrage
2,area_to_lanczos,0.00000472946272073911,0.00000301916291133442,0.00000849470890635072,0.00000564034762608672,0.00000615643205443298,0.00000560802284378879
0,area_to_bicubic,0.00000508675241493998,0.00000302277501656234,0.00000849977670427555,0.00000542576994580762,0.00000622245009865779,0.00000565150483604865
1,area_to_bilinear,0.00000531454335122522,0.00000317977796440521,0.00000926449863087728,0.00000634526107403220,0.00000725185422952262,0.00000627118705001251
10,bilinear_to_lanczos,0.00000614913820393779,0.00000370716240671095,0.00000839666913621656,0.00000664499351279056,0.00000678391667187924,0.00000633637598630702
9,bilinear_to_bicubic,0.00000631699649082412,0.00000360961729210539,0.00000863615718060248,0.00000647454025103012,0.00000666932988422645,0.00000634132821975771
14,lanczos_to_bilinear,0.00000561487646446536,0.00000361063212758896,0.00000813105695894008,0.00000654101588608010,0.00000811246644837755,0.00000640200957709041
5,bicubic_to_bilinear,0.00000641136070851624,0.00000355429597935354,0.00000870042556822735,0.00000643010484504711,0.00000755243692612348,0.00000652972480545354
13,lanczos_to_bicubic,0.00000589942717816057,0.00000485250872887798,0.00000823491519963310,0.00000660061707523938,0.00000776606846912784,0.00000667070733020777
6,bicubic_to_lanczos,0.00000673500659641990,0.00000397130461968740,0.00000893300149811913,0.00000674403079749851,0.00000747140816424929,0.00000677095033519485
3,area_to_nearest,0.00000578161931964355,0.00000427385269738276,0.00000973123103133449,0.00000777580404597276,0.00001069977774001151,0.00000765245696686901


In [6]:
# 128x128
import pandas as pd
pd.set_option('display.float_format', '{:.20f}'.format)
file_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Interpolation\Comparision_128x128\mean_value_of_the_intenstiy_difference_128x128.csv'

dataframe = pd.read_csv(file_path)
dataframe['Averrage'] = dataframe.iloc[:, 1:].mean(axis=1)
dataframe.sort_values(by='Averrage').head(25)

,Method,Image 1,Image 2,Image 3,Image 4,Image 5,Averrage
2,area_to_lanczos,0.00000497323035532499,0.00000223688923391458,0.00000735114845396424,0.00000334665574517357,0.00000377862074083532,0.00000433730890584254
0,area_to_bicubic,0.00000507126539023702,0.00000242544840128843,0.00000738197167808763,0.00000348585472636244,0.00000399964175637856,0.00000447283639047081
9,bilinear_to_bicubic,0.00000543727869775631,0.00000303215606287036,0.00000622272274116070,0.00000377111418478323,0.00000413330852442989,0.00000451931604220010
10,bilinear_to_lanczos,0.00000563470547480797,0.00000310002252222515,0.00000624687612474978,0.00000376588952244933,0.00000391888843348104,0.00000453327641554265
1,area_to_bilinear,0.00000511153232242399,0.00000241150328078505,0.00000729202950204597,0.00000376016693156241,0.00000477798208455905,0.00000467064282427529
14,lanczos_to_bilinear,0.00000571723387555149,0.00000221836034390501,0.00000721098394229834,0.00000429545536280463,0.00000460292413290726,0.00000480899153149334
3,area_to_nearest,0.00000633029532227228,0.00000256132688878456,0.00000674097043253207,0.00000364382591246459,0.00000490783562025008,0.00000483685083526072
13,lanczos_to_bicubic,0.00000603028458276376,0.00000240634787769894,0.00000739351116873608,0.00000406240039899609,0.00000429486424046527,0.00000483748165373203
5,bicubic_to_bilinear,0.00000568424832513028,0.00000278065042594740,0.00000648111673154392,0.00000412836229944638,0.00000521135330872372,0.00000485714621815834
6,bicubic_to_lanczos,0.00000614584478819656,0.00000287645488882723,0.00000714852902618252,0.00000439406204837246,0.00000534019671654396,0.00000518101749362455


In [11]:
# 256x256
import pandas as pd
pd.set_option('display.float_format', '{:.20f}'.format)
file_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Interpolation\Comparision_256x256\mean_value_of_the_intenstiy_difference_256x256.csv'
dataframe = pd.read_csv(file_path)
dataframe['Averrage'] = dataframe.iloc[:, 1:].mean(axis=1)
dataframe.sort_values(by='Averrage').head(25)

,Method,Image 1,Image 2,Image 3,Image 4,Image 5,Averrage
4,bicubic_to_area,0.00000509416868480766,0.00000214220367946653,0.00000300406608342514,0.00000158760579609597,0.00000233718417366323,0.00000283304568349170
7,bicubic_to_nearest,0.00000509416868480766,0.00000214220367946653,0.00000300406608342514,0.00000158760579609597,0.00000233718417366323,0.00000283304568349170
15,lanczos_to_nearest,0.00000405026482247723,0.00000129168273823121,0.00000383125349357197,0.00000173798837598934,0.00000343912925881756,0.00000287006373781746
12,lanczos_to_area,0.00000405026482247723,0.00000129168273823121,0.00000383125349357197,0.00000173798837598934,0.00000343912925881756,0.00000287006373781746
3,area_to_nearest,0.00000558208308397326,0.00000220329502689200,0.00000473338721446529,0.00000152113741235705,0.00000257189339448043,0.00000332235922643361
8,bilinear_to_area,0.00000558208308397326,0.00000220329502689200,0.00000473338721446529,0.00000152113741235705,0.00000257189339448043,0.00000332235922643361
11,bilinear_to_nearest,0.00000558208308397326,0.00000220329502689200,0.00000473338721446529,0.00000152113741235705,0.00000257189339448043,0.00000332235922643361
6,bicubic_to_lanczos,0.00000651737838953912,0.00000199165622092062,0.00000498859254822853,0.00000179296201258573,0.00000198691795364985,0.00000345550142498477
5,bicubic_to_bilinear,0.00000625341636760491,0.00000216608505909534,0.00000499780803184896,0.00000188944596826513,0.00000221251521428859,0.00000350385412822058
13,lanczos_to_bicubic,0.00000536271871455933,0.00000130959528515946,0.00000585267533942220,0.00000187018816669171,0.00000316355855832547,0.00000351174721283163


# Interplolate an image according to the best method

In [14]:
import cv2
import os

input_image_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Interpolation\Test_Bild\original_phasemask.png'  
output_folder = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Interpolation\Test_Bild/' 
image = cv2.imread(input_image_path)

# Definiere die Größen und die entsprechenden Downsampling- und Upsampling-Methoden
sizes_methods = [
    (64, cv2.INTER_AREA, cv2.INTER_LANCZOS4),  
    (128, cv2.INTER_AREA, cv2.INTER_LANCZOS4), 
    (256, cv2.INTER_CUBIC, cv2.INTER_NEAREST)  
]

# Erstelle den Ausgabeordner, falls er nicht existiert
os.makedirs(output_folder, exist_ok=True)

for size, downsampling_method, upsampling_method in sizes_methods:
    # Downsampling auf die angegebene Größe
    resized_image = cv2.resize(image, (size, size), interpolation=downsampling_method)
    
    # Upsampling zurück auf 512x512
    resized_back_image = cv2.resize(resized_image, (512, 512), interpolation=upsampling_method)
    
    # Speichern des Bildes mit dem entsprechenden Dateinamen
    output_path = os.path.join(output_folder, f'{size}_to_512.png')
    cv2.imwrite(output_path, resized_back_image)

    print(f'Bild wurde auf {size}x{size} und dann zurück auf 512x512 skaliert und als {output_path} gespeichert.')


SyntaxError: unterminated string literal (detected at line 5) (4185032264.py, line 5)